In [14]:
from keras import applications
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
from keras.models import Sequential, Model 
from keras.layers import Dropout, Flatten, Dense, GlobalAveragePooling2D
from keras import backend as k 
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, TensorBoard, EarlyStopping, ReduceLROnPlateau

%matplotlib inline
import matplotlib.pyplot as plt

img_width, img_height = 224,224
train_data_dir = "train"
validation_data_dir = "val"
batch_size = 16
epochs = 50

# model = applications.densenet.DenseNet121(include_top=True, weights='imagenet', classes=1000)
model = applications.densenet.DenseNet121(include_top=True, weights='imagenet', input_shape = (img_width, img_height, 3), classes=1000)
model.layers.pop()

for layer in model.layers:
    layer.trainable = False

#Adding custom Layers 
x = model.layers[-1].output
predictions = Dense(2, activation="softmax")(x)

# creating the final model 
model_final = Model(inputs = model.input, outputs = predictions)

# compile the model 
model_final.compile(loss = "categorical_crossentropy", optimizer = optimizers.Adam(lr=0.001), metrics=["accuracy"])
model_final.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_8 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
zero_padding2d_15 (ZeroPadding2 (None, 230, 230, 3)  0           input_8[0][0]                    
__________________________________________________________________________________________________
conv1/conv (Conv2D)             (None, 112, 112, 64) 9408        zero_padding2d_15[0][0]          
__________________________________________________________________________________________________
conv1/bn (BatchNormalization)   (None, 112, 112, 64) 256         conv1/conv[0][0]                 
__________________________________________________________________________________________________
conv1/relu

In [15]:
train_datagen = ImageDataGenerator(
rescale = 1./255,
horizontal_flip = True,
fill_mode = "nearest",
zoom_range = 0.3,
width_shift_range = 0.3,
height_shift_range=0.3,
rotation_range=30)

test_datagen = ImageDataGenerator(
rescale = 1./255)

train_generator = train_datagen.flow_from_directory(
train_data_dir,
target_size = (img_height, img_width),
batch_size = batch_size, 
class_mode = "categorical")

validation_generator = test_datagen.flow_from_directory(
validation_data_dir,
target_size = (img_height, img_width),
class_mode = "categorical")

nb_train_samples = train_generator.samples
nb_validation_samples = validation_generator.samples

# Save the model according to the conditions  
checkpoint = ModelCheckpoint("densenet.h5", monitor='val_loss', verbose=1, save_best_only=True, save_weights_only=False, mode='auto', period=1)
early = EarlyStopping(monitor='val_loss', min_delta=0, patience=10, verbose=1, mode='auto')
plateau = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2)


Found 227178 images belonging to 2 classes.
Found 11446 images belonging to 2 classes.


In [16]:

# Train the model 
model_final.fit_generator(
train_generator,
samples_per_epoch = nb_train_samples,
epochs = epochs,
steps_per_epoch = int(nb_train_samples / batch_size),
validation_data = validation_generator,
validation_steps = int(nb_validation_samples / batch_size),
callbacks = [checkpoint, early, plateau])

/anaconda/envs/py35/lib/python3.5/site-packages/ipykernel_launcher.py:10: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<keras.pre..., validation_data=<keras.pre..., steps_per_epoch=14198, callbacks=[<keras.ca..., epochs=50, validation_steps=715)`
  # Remove the CWD from sys.path while we load stuff.


Epoch 1/50
   26/14198 [..............................] - ETA: 13:08:28 - loss: 0.4281 - acc: 0.8269

KeyboardInterrupt: 